In [1]:
import pandas as pd
ratings_columns = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('u.data', sep='\t', names=ratings_columns, encoding='latin-1')
ratings.drop( "timestamp", inplace = True, axis = 1 )
print ratings.shape
ratings.head(10)

(100000, 3)


,user_id,movie_id,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1
5,298,474,4
6,115,265,2
7,253,465,5
8,305,451,3
9,6,86,3


In [2]:
user_movies = ratings.pivot( index='user_id', columns='movie_id', values = "rating" ).reset_index(drop=True)
user_movies.fillna( 0, inplace = True )
user_movies=pd.DataFrame(user_movies)
print user_movies.shape
user_movies.head()

(943, 1682)


movie_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
0,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
from sklearn.metrics.pairwise import pairwise_distances
users_similarity = 1 - pairwise_distances( user_movies.as_matrix(), metric="correlation" )
users_similarity_df = pd.DataFrame( users_similarity )
print users_similarity.shape
users_similarity_df.head()

(943, 943)


,0,1,2,3,4,5,6,7,8,9,...,933,934,935,936,937,938,939,940,941,942
0,1.000000,0.106322,-0.016424,0.021716,0.302592,0.345976,0.320017,0.276208,0.038812,0.288491,...,0.285923,0.068878,0.189272,0.146979,0.119444,0.060175,0.247483,0.115504,0.108984,0.323231
1,0.106322,1.000000,0.082680,0.160323,0.020218,0.197844,0.022886,0.072772,0.143716,0.106861,...,0.106416,0.288550,0.325233,0.408466,0.289112,0.204094,0.190848,0.144177,0.138292,0.054235
2,-0.016424,0.082680,1.000000,0.331674,-0.028322,0.017673,-0.013384,0.055385,0.043505,0.012080,...,-0.020605,0.018746,0.124174,0.046247,0.088974,-0.001145,0.127539,0.084532,0.102025,-0.023601
3,0.021716,0.160323,0.331674,1.000000,-0.002426,0.030206,0.041056,0.171116,0.089372,0.024018,...,0.016641,0.019738,0.105135,0.179598,0.122649,0.010890,0.174904,0.140811,0.149575,0.025367
4,0.302592,0.020218,-0.028322,-0.002426,1.000000,0.153409,0.279677,0.209280,0.025496,0.117166,...,0.272866,0.039333,0.013999,0.039519,0.085548,0.024292,0.182466,0.112571,0.096603,0.248991


In [4]:
from sklearn.neighbors import NearestNeighbors

def find_neighborhood(user_id, n):

    model_knn = NearestNeighbors(metric = "correlation", algorithm = "brute") 
    model_knn.fit(user_movies)
    distances, indices = model_knn.kneighbors(user_movies.iloc[user_id-1, :].values.reshape(1, -1), n_neighbors = n+1)
    similarities = 1-distances.flatten()
    print '{0} most similar users for user with id {1}:\n'.format(n, user_id)
    
    for i in range(0, len(indices.flatten())):
        # pomiń, jeśli ten sam użytkownik
        if indices.flatten()[i]+1 == user_id: 
            continue;

        else:
            print '{0}: User {1}, with similarity of {2}'.format(i, indices.flatten()[i]+1, similarities.flatten()[i])
    
    return similarities,indices


In [5]:
find_neighborhood(1, 10)

10 most similar users for user with id 1:

1: User 916, with similarity of 0.487949133033
2: User 738, with similarity of 0.471104840007
3: User 864, with similarity of 0.465282748572
4: User 457, with similarity of 0.457111312198
5: User 268, with similarity of 0.457104969265
6: User 823, with similarity of 0.454805111853
7: User 92, with similarity of 0.445429698702
8: User 514, with similarity of 0.44535899324
9: User 435, with similarity of 0.444814676191
10: User 521, with similarity of 0.441489570437


(array([1.        , 0.48794913, 0.47110484, 0.46528275, 0.45711131,
        0.45710497, 0.45480511, 0.4454297 , 0.44535899, 0.44481468,
        0.44148957]),
 array([[  0, 915, 737, 863, 456, 267, 822,  91, 513, 434, 520]]))

In [6]:
import numpy as np
def predict_rate(user_id, item_id, n):
    similarities, indices=find_neighborhood(user_id, n)
    neighborhood_ratings =[]
    
    for i in range(0, len(indices.flatten())):
        if indices.flatten()[i]+1 == user_id:
            continue;
        else: 
            neighborhood_ratings.append(user_movies.iloc[indices.flatten()[i],item_id-1])
    
    
    weights = np.delete(indices.flatten(), 0) #delete weight for input user
    prediction = round((neighborhood_ratings * weights).sum() / weights.sum())

    print '\nPredicted rating for user {0} -> item {1}: {2}'.format(user_id,item_id,prediction)


In [7]:
predict_rate(1, 10, 30)

30 most similar users for user with id 1:

1: User 916, with similarity of 0.487949133033
2: User 738, with similarity of 0.471104840007
3: User 864, with similarity of 0.465282748572
4: User 457, with similarity of 0.457111312198
5: User 268, with similarity of 0.457104969265
6: User 823, with similarity of 0.454805111853
7: User 92, with similarity of 0.445429698702
8: User 514, with similarity of 0.44535899324
9: User 435, with similarity of 0.444814676191
10: User 521, with similarity of 0.441489570437
11: User 886, with similarity of 0.433400328476
12: User 387, with similarity of 0.430556467094
13: User 889, with similarity of 0.429917759106
14: User 933, with similarity of 0.42613659934
15: User 297, with similarity of 0.425869422463
16: User 44, with similarity of 0.423730391039
17: User 429, with similarity of 0.423702761867
18: User 715, with similarity of 0.423656242299
19: User 64, with similarity of 0.422094324927
20: User 303, with similarity of 0.418509369673
21: User 30